In [5]:
import pandas as pd
import numpy as np
import pickle
import sys
sys.path.append("../")
import rsc.Resources as rsc
from selenium import webdriver
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from fuzzywuzzy import process, fuzz
import warnings
warnings.filterwarnings('ignore')

In [6]:
#Here i the csv that ill be using.
df_movies = pd.read_csv("../Data/imdb_top_1000.csv")

#I import the list that i scrapped with seleneum thats saved in a pickle so i dont lose the info when the kernel shuts down.
with open("../Data/libros_scrap.pkl", "rb") as alias:
    libro2 =pickle.load(alias)

In [7]:
#Seleneum configurations.

opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window

In [ ]:
#This loop runs my seleneum with its function.
libro = []
resultado = []
for i in df_movies["Series_Title"]:
    rsc.busqueda(i,resultado)
    print(resultado)
    libro.append(resultado)
    resultado = []

In [8]:
#Here i run all of my cleaning functions.
scrap_limpio = rsc.listas_libros(libro2)
lista_votos_limpio = [rsc.limpieza_votos(voto) for voto in scrap_limpio[2]]
nota_promedio_limpio = [rsc.promedio(rati) for rati in scrap_limpio[1]]
Autor_limpio = [rsc.Autor(auto) for auto in scrap_limpio[0]]
fuzzy_titulo = rsc.semejanza(scrap_limpio[3],list(df_movies["Series_Title"]))
df_movies["Gross"] = df_movies["Gross"].str.replace(',','')

In [9]:
#I create columns in my movie DataFrame so i can then match movies with books
df_movies["Existe_libro"] = fuzzy_titulo
df_movies["Autor_libro"] = Autor_limpio
df_movies["Nota_promedio_libro"] = nota_promedio_limpio
df_movies["Votos_libro"] = lista_votos_limpio
df_movies['Nota_promedio_libro'] = df_movies['Nota_promedio_libro'].apply(lambda x: x*2)

In [10]:
#I create a new DataFrame with only the rows that match that there is an existing book.
df_movies_limpio = df_movies[df_movies["Existe_libro"] == True]

#Here i copy all of the columns that i will use.
df_libros = df_movies_limpio[["Autor_libro","Nota_promedio_libro","Votos_libro"]].copy()

#And here i eliminate all of the columns that i won't be using anymore.
df_movies_limpio.drop("Existe_libro",inplace=True,axis=1),
#df_movies_limpio.drop("Nota_promedio_libro",inplace=True,axis=1)
#df_movies_limpio.drop("Votos_libro",inplace=True,axis=1)
#df_movies_limpio.drop("Autor_libro",inplace=True,axis=1)



(None,)

In [11]:
#And finally i create two new CSV with all of the data that ill be for this project.
df_movies_limpio.to_csv(f"../Data/df_movies_limpio.csv")
df_libros.to_csv(f"../Data/df_libros.csv")